In [104]:
import pandas as pd
from os.path import isfile
from tailer import tail
from FinanceDataReader import DataReader
from pykrx import stock
from time import sleep, time


from pattern_labelling import * 


nsq_p = pd.read_csv("temp/nasdaq.csv")[['date','nasdaq']]
nsq_p['date'] = pd.to_datetime(nsq_p['date'])

In [53]:
stock_list = pd.read_csv("../resources/stockcode.csv", dtype = {"종목코드": str, "회사명": str})

In [70]:
# to_csv performance test
df = pd.read_csv("temp/000020.csv",parse_dates=['date'], index_col=[0])

startTime = time()
for i in range(10000):
    df.to_csv(f"temp/test_{i}.csv")
    
endTime = time()
print("time: {}ms".format((endTime - startTime)*1000))

time: 211903.57327461243ms


In [71]:
# read_csv performance test

startTime = time()
for i in range(10000):
    df = pd.read_csv("temp/000020.csv",parse_dates=['date'], index_col=[0])
    
endTime = time()
print("time: {}ms".format((endTime - startTime)*1000))

time: 56258.91304016113ms


----

In [ ]:
    first_df = pd.read_csv('resources/ohlcv_p1p2p3_nasdq/950200.csv', parse_dates=['date'], index_col=[0])

    pd_last_date = first_df['date'].iloc[-1]
    pd_today = pd.to_datetime(pd.Timestamp.now().date())
    pd_drange = pd.date_range(pd_last_date, pd_today)   # 데이터를 갱신할 날짜 범위

    for pd_date in pd_drange:
        df_update = stock.get_market_ohlcv_by_ticker(pd_date)
        if not len(df_update):  # 장이 열리지 않은 날이면 skip
            continue
        df_update = df_update.reset_index()
        for stock_code in stock_list['종목코드'].iloc:
            try:
                stock_ = df_update[df_update['종목코드']=='999999'].values[0]
            except IndexError:
                stock_ = [stock_code, 0,0,0,0,0,0]
            filename = f'resources/ohlcv_p1p2p3_nasdq/{stock_code}.csv'
            try:
                update_target = pd.read_csv(filename, parse_dates=['date'], index_col=[0])
            except FileNotFoundError:
                break
            if update_target['date'].values[-1] > pd_date:
                continue    # 이전에 오류나서 이미 일부가 업데이트된 상태인 경우, 넘김
            elif update_target['date'].values[-1] == pd_date:
                update_target['open'].values[-1] = stock_['시가']
                update_target['high'].values[-1] = stock_['고가']
                update_target['low'].values[-1] = stock_['저가']
                update_target['close'].values[-1] = stock_['종가']
                update_target['volume'].values[-1] = stock_['거래량']
                flag = 0
            else:
                update_target = update_target.append({'date':pd_date,'open':stock_['시가'],'high':stock_['고가'],'low':stock_['저가'],'close':stock_['종가'],'volume':stock_['거래량']}, 
                                    ignore_index=True)
                flag = 1

            last_idx = len(update_target)-1
            update_target['pattern1'].values[last_idx] = labellingD0(update_target.iloc[last_idx])
            update_target['pattern2'].values[last_idx] = labellingD1(update_target.iloc[last_idx-1:last_idx+1])
            update_target['pattern3'].values[last_idx] = labellingD2(update_target.iloc[last_idx-2:last_idx+1])
            try:
                update_target['nasdaq'].values[last_idx] = nsq_p[nsq_p['date']==pd_date].values[0][1]
            except IndexError:
                raise Exception("call_nasdaq()부터 실행하세요.")

            if flag:
                # modify file
                temp = update_target.values[last_idx]
                temp[0] = temp[0].date()
                for i in range(6):
                    temp[i] = str(temp[i])
                temp = "".join([temp[i//2] if i%2==0 else "," for i in range(19)])
                temp += '\n'
                with open(filename,'a',encoding='UTF-8') as f:
                    f.write(temp)
            else:
                update_target.to_csv(filename)

In [141]:
first_df = pd.read_csv('../resources/ohlcv_p1p2p3_nasdq/950200.csv', parse_dates=['date'], index_col=[0])

In [142]:
    pd_last_date = first_df['date'].iloc[-1]
    pd_today = pd.to_datetime(pd.Timestamp.now().date())
    pd_drange = pd.date_range(pd_last_date, pd_today)   # 데이터를 갱신할 날짜 범위

In [143]:
pd_date = pd_drange[0]

Timestamp('2020-10-27 00:00:00', freq='D')

In [144]:
df_update = stock.get_market_ohlcv_by_ticker(pd_date)
df_update = df_update.reset_index()

,종목코드,종목명,시가,고가,저가,종가,거래량,거래대금,시가총액,시총비중,상장주식수
0,000020,동화약품,16850,18150,15900,17500,854079,14764308850,488800725000,0.029999,27931470
1,000040,KR모터스,944,970,921,923,1373435,1288723684,84603119614,0.000000,91661018
2,000050,경방,10700,10950,10650,10900,18826,203234700,298826443000,0.020004,27415270
3,000060,메리츠화재,14950,15150,14500,14650,330214,4894050450,1665412000000,0.090027,113680000
4,000070,삼양홀딩스,61200,62300,61200,62000,8836,547195200,530984802000,0.029999,8564271
...,...,...,...,...,...,...,...,...,...,...,...
2499,950160,코오롱티슈진,0,0,0,8010,0,0,489551575500,0.029999,61117550
2500,950170,JTC,4215,4290,4145,4200,48656,204350725,147023171400,0.010002,35005517
2501,950180,SNK,12400,12600,12100,12500,37169,457708100,263272500000,0.010002,21061800
2502,950190,미투젠,26700,27250,26450,26750,30228,811547800,344796586000,0.020004,12889592


In [145]:
stock_code = stock_list['종목코드'].iloc[0]

'000020'

In [173]:
stock_ = df_update[df_update['종목코드']==stock_code].values[0]
filename = f'temp/{stock_code}.csv'
stock_

array(['000020', '동화약품', 16850, 18150, 15900, 17500, 854079, 14764308850,
       488800725000, 0.029998779296875, 27931470], dtype=object)

In [176]:

filename = f'temp/{stock_code}.csv'
stock_

['000020', 0, 0, 0, 0, 0, 0]

pandas.core.frame.DataFrame

In [177]:
update_target = pd.read_csv(filename, parse_dates=['date'], index_col=[0])
update_target = update_target.append({'date':pd_date,'open':stock_[2],'high':stock_[3],'low':stock_[4],'close':stock_[5],'volume':stock_[6]}, 
                                    ignore_index=True)
update_target

,date,open,high,low,close,volume,pattern1,pattern2,pattern3,nasdaq
0,2012-01-02,4070,4100,4060,4085,25451,K00,NaN,NaN,NaN
1,2012-01-03,4100,4160,4100,4130,45291,P03,K00P03,NaN,NaN
2,2012-01-04,4135,4160,4125,4150,42911,K00,P00K00,P10K00,U04
3,2012-01-05,4150,4290,4110,4270,85140,P05,K00P05,P10P05,T01
4,2012-01-06,4300,4325,4220,4240,45535,M05,P00M05,P10M05,U04
...,...,...,...,...,...,...,...,...,...,...
2166,2020-10-22,19850,20050,19000,19200,620779,M05,M00M05,M10M05,D04
2167,2020-10-23,19500,20600,19350,19800,563966,P05,M00P05,M10P05,U03
2168,2020-10-26,19500,19750,17000,17100,1338181,M15,P00M15,S01M15,U04
2169,2020-10-27,16850,18150,15900,17500,851662,P05,M10P05,S07P05,D04


In [181]:
last_idx

2170